# SpaceX dashboard

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html #from dash import html ->así debe ser porque 'dash_html_components' está obsoleto
import dash_core_components as dcc #from dash import dcc -> así debe ser porque 'dash_core_components' está obsoleto
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
options_dropdown_menu=[ {'label': site, 'value': site} for site in list(spacex_df['Launch Site'].unique())]
options_dropdown_menu.append({'label': 'All Sites', 'value': 'ALL'})  
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                  dcc.Dropdown(id='site-dropdown',
                                                options=options_dropdown_menu,
                                                value='ALL',
                                                placeholder="Select launch site here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='All sites success rate')
        return fig
    else:
        # return the outcomes piechart for a selected site
        success_counts = filtered_df['class'].value_counts()
        fig = px.pie(filtered_df, values=success_counts.values, 
        names=success_counts.index, 
        title=f'{entered_site} success rate')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider',component_property='value'))
def get_scatter_plot(entered_site, payload_range):
    filtered_df = spacex_df[ (payload_range[0]< spacex_df['Payload Mass (kg)']) & (payload_range[1]> spacex_df['Payload Mass (kg)']) ]
    if entered_site=='ALL':
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class",
                        color='Booster Version Category',
                        title='All Sites') #title='All Sites'
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class",
                        color='Booster Version Category',
                        title=f'{entered_site}') #title=f'{entered_site}'
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()